<a href="https://colab.research.google.com/github/sjaanus97/P07-fast_growing_comp/blob/main/fast_growing_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob
import numpy as np
from functools import reduce
import time
import datetime
import math
pd.set_option("display.max_columns", None)

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/file/d/18B4656TSdWoJfqP3LE56nrg7QyiMNmJR'

downloaded = drive.CreateFile({'id':'18B4656TSdWoJfqP3LE56nrg7QyiMNmJR'}) 
downloaded.GetContentFile('input.zip')

In [ ]:
!unzip input.zip

Archive:  input.zip
   creating: input/
  inflating: input/maksuvolglaste_nimekiri.xlsx  
   creating: input/tasutud_maksud/
  inflating: input/tasutud_maksud/tasutud_maksud_2017_iii_kv.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2017_ii_kv.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2017_iv_kv.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2017_i_kv.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2018_iii_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2018_ii_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2018_iv_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2018_i_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2019_iii_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2019_ii_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2019_iv_kvartal.xlsx  
  inflating: input/tasutud_maksud/tasutud_maksud_2019_i_kvartal.xlsx  
  inflating: input/tasutud_maks

In [ ]:
path = 'input/tasutud_maksud' # use your path
all_files = glob.glob(path + "/*.xlsx")
all_files.sort()

# Preprocess

In [ ]:
def mod_df(path):
    file_name = path.split("/")[-1]
    file_name = file_name.split("\\")[-1]
    year = file_name.split("_")[2]
    quater = file_name.split("_")[3]

    suffix = "_{0}_{1}".format(year,quater)
    
    df = pd.read_excel(path,usecols=['Nimi', 'Registrikood','Riiklikud Maksud', 'Tööjõumaksud Ja Maksed', 'Kaive', 'Tootajaid' ], dtype={'Registrikood': str}) #nrows=1000
    #df['date'] = year
    #df['type'] = "fast_grower"
    df = df.rename(columns={'Nimi':"Nimi", 'Registrikood': "Registrikood", 'Riiklikud Maksud': 'Riiklikud Maksud'+suffix, 'Tööjõumaksud Ja Maksed': 'Tööjõumaksud Ja Maksed'+suffix, 'Kaive':'Kaive'+suffix, 'Tootajaid':'Tootajaid'+suffix,
                           "date": "date"+suffix, 'type':'type'+suffix})
    return df



In [ ]:
#all_files = all_files[6:10] # remove this row later
df_from_each_file = (mod_df(f) for f in all_files)

df_merged = reduce(lambda  left,right: pd.concat([pd.merge(left,right,on=['Registrikood', 'Nimi'], #remove concat, if you want every company in 1 row
                                            how='outer'), right]), df_from_each_file)

df_merged = df_merged.drop_duplicates()
#result = df_merged.copy()

In [ ]:
print(df_merged.shape)
print(df_merged.columns)
df_merged.sample(n = 3)

(2538009, 78)
Index(['Registrikood', 'Nimi', 'Riiklikud Maksud_2017_i',
       'Tööjõumaksud Ja Maksed_2017_i', 'Kaive_2017_i', 'Tootajaid_2017_i',
       'Riiklikud Maksud_2017_ii', 'Tööjõumaksud Ja Maksed_2017_ii',
       'Kaive_2017_ii', 'Tootajaid_2017_ii', 'Riiklikud Maksud_2017_iii',
       'Tööjõumaksud Ja Maksed_2017_iii', 'Kaive_2017_iii',
       'Tootajaid_2017_iii', 'Riiklikud Maksud_2017_iv',
       'Tööjõumaksud Ja Maksed_2017_iv', 'Kaive_2017_iv', 'Tootajaid_2017_iv',
       'Riiklikud Maksud_2018_i', 'Tööjõumaksud Ja Maksed_2018_i',
       'Kaive_2018_i', 'Tootajaid_2018_i', 'Riiklikud Maksud_2018_ii',
       'Tööjõumaksud Ja Maksed_2018_ii', 'Kaive_2018_ii', 'Tootajaid_2018_ii',
       'Riiklikud Maksud_2018_iii', 'Tööjõumaksud Ja Maksed_2018_iii',
       'Kaive_2018_iii', 'Tootajaid_2018_iii', 'Riiklikud Maksud_2018_iv',
       'Tööjõumaksud Ja Maksed_2018_iv', 'Kaive_2018_iv', 'Tootajaid_2018_iv',
       'Riiklikud Maksud_2019_i', 'Tööjõumaksud Ja Maksed_2019_i',
    

,Registrikood,Nimi,Riiklikud Maksud_2017_i,Tööjõumaksud Ja Maksed_2017_i,Kaive_2017_i,Tootajaid_2017_i,Riiklikud Maksud_2017_ii,Tööjõumaksud Ja Maksed_2017_ii,Kaive_2017_ii,Tootajaid_2017_ii,Riiklikud Maksud_2017_iii,Tööjõumaksud Ja Maksed_2017_iii,Kaive_2017_iii,Tootajaid_2017_iii,Riiklikud Maksud_2017_iv,Tööjõumaksud Ja Maksed_2017_iv,Kaive_2017_iv,Tootajaid_2017_iv,Riiklikud Maksud_2018_i,Tööjõumaksud Ja Maksed_2018_i,Kaive_2018_i,Tootajaid_2018_i,Riiklikud Maksud_2018_ii,Tööjõumaksud Ja Maksed_2018_ii,Kaive_2018_ii,Tootajaid_2018_ii,Riiklikud Maksud_2018_iii,Tööjõumaksud Ja Maksed_2018_iii,Kaive_2018_iii,Tootajaid_2018_iii,Riiklikud Maksud_2018_iv,Tööjõumaksud Ja Maksed_2018_iv,Kaive_2018_iv,Tootajaid_2018_iv,Riiklikud Maksud_2019_i,Tööjõumaksud Ja Maksed_2019_i,Kaive_2019_i,Tootajaid_2019_i,Riiklikud Maksud_2019_ii,Tööjõumaksud Ja Maksed_2019_ii,Kaive_2019_ii,Tootajaid_2019_ii,Riiklikud Maksud_2019_iii,Tööjõumaksud Ja Maksed_2019_iii,Kaive_2019_iii,Tootajaid_2019_iii,Riiklikud Maksud_2019_iv,Tööjõumaksud Ja Maksed_2019_iv,Kaive_2019_iv,Tootajaid_2019_iv,Riiklikud Maksud_2020_i,Tööjõumaksud Ja Maksed_2020_i,Kaive_2020_i,Tootajaid_2020_i,Riiklikud Maksud_2020_ii,Tööjõumaksud Ja Maksed_2020_ii,Kaive_2020_ii,Tootajaid_2020_ii,Riiklikud Maksud_2020_iii,Tööjõumaksud Ja Maksed_2020_iii,Kaive_2020_iii,Tootajaid_2020_iii,Riiklikud Maksud_2020_iv,Tööjõumaksud Ja Maksed_2020_iv,Kaive_2020_iv,Tootajaid_2020_iv,Riiklikud Maksud_2021_i,Tööjõumaksud Ja Maksed_2021_i,Kaive_2021_i,Tootajaid_2021_i,Riiklikud Maksud_2021_ii,Tööjõumaksud Ja Maksed_2021_ii,Kaive_2021_ii,Tootajaid_2021_ii,Riiklikud Maksud_2021_iii,Tööjõumaksud Ja Maksed_2021_iii,Kaive_2021_iii,Tootajaid_2021_iii
1333086,12876186,SKOOP3 STUUDIO OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471.11,1198.28,7793.83,2.0,2938.86,1236.42,10273.79,2.0,2209.91,1338.76,6289.52,2.0,1637.84,1236.42,7891.1,NaN,4556.95,1260.96,17073.33,2.0,3967.57,1891.44,12201.2,2.0,2660.94,1891.44,5495.5,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN
2470385,14330008,B.V.METALL OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3479.14,1875.51,3020.84,NaN,5077.71,1875.51,11334.99,NaN
1239694,12712970,JG RÖR OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.97,155.97,1070.0,4.0,42.65,27.33,1774.00,4.0,77.26,0.00,0.00,NaN


In [ ]:
df_merged.loc[df_merged['Registrikood'] == '14339972']

,Registrikood,Nimi,Riiklikud Maksud_2017_i,Tööjõumaksud Ja Maksed_2017_i,Kaive_2017_i,Tootajaid_2017_i,Riiklikud Maksud_2017_ii,Tööjõumaksud Ja Maksed_2017_ii,Kaive_2017_ii,Tootajaid_2017_ii,Riiklikud Maksud_2017_iii,Tööjõumaksud Ja Maksed_2017_iii,Kaive_2017_iii,Tootajaid_2017_iii,Riiklikud Maksud_2017_iv,Tööjõumaksud Ja Maksed_2017_iv,Kaive_2017_iv,Tootajaid_2017_iv,Riiklikud Maksud_2018_i,Tööjõumaksud Ja Maksed_2018_i,Kaive_2018_i,Tootajaid_2018_i,Riiklikud Maksud_2018_ii,Tööjõumaksud Ja Maksed_2018_ii,Kaive_2018_ii,Tootajaid_2018_ii,Riiklikud Maksud_2018_iii,Tööjõumaksud Ja Maksed_2018_iii,Kaive_2018_iii,Tootajaid_2018_iii,Riiklikud Maksud_2018_iv,Tööjõumaksud Ja Maksed_2018_iv,Kaive_2018_iv,Tootajaid_2018_iv,Riiklikud Maksud_2019_i,Tööjõumaksud Ja Maksed_2019_i,Kaive_2019_i,Tootajaid_2019_i,Riiklikud Maksud_2019_ii,Tööjõumaksud Ja Maksed_2019_ii,Kaive_2019_ii,Tootajaid_2019_ii,Riiklikud Maksud_2019_iii,Tööjõumaksud Ja Maksed_2019_iii,Kaive_2019_iii,Tootajaid_2019_iii,Riiklikud Maksud_2019_iv,Tööjõumaksud Ja Maksed_2019_iv,Kaive_2019_iv,Tootajaid_2019_iv,Riiklikud Maksud_2020_i,Tööjõumaksud Ja Maksed_2020_i,Kaive_2020_i,Tootajaid_2020_i,Riiklikud Maksud_2020_ii,Tööjõumaksud Ja Maksed_2020_ii,Kaive_2020_ii,Tootajaid_2020_ii,Riiklikud Maksud_2020_iii,Tööjõumaksud Ja Maksed_2020_iii,Kaive_2020_iii,Tootajaid_2020_iii,Riiklikud Maksud_2020_iv,Tööjõumaksud Ja Maksed_2020_iv,Kaive_2020_iv,Tootajaid_2020_iv,Riiklikud Maksud_2021_i,Tööjõumaksud Ja Maksed_2021_i,Kaive_2021_i,Tootajaid_2021_i,Riiklikud Maksud_2021_ii,Tööjõumaksud Ja Maksed_2021_ii,Kaive_2021_ii,Tootajaid_2021_ii,Riiklikud Maksud_2021_iii,Tööjõumaksud Ja Maksed_2021_iii,Kaive_2021_iii,Tootajaid_2021_iii
1905164,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,97414.16,NaN,7937.0,0.0,195359.07,NaN,8407.35,0.0,188152.72,NaN,12015.54,0.0,198880.19,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905166,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97414.16,NaN,7937.0,0.0,195359.07,NaN,8407.35,0.0,188152.72,NaN,12015.54,0.0,198880.19,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905167,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7937.0,0.0,195359.07,NaN,8407.35,0.0,188152.72,NaN,12015.54,0.0,198880.19,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905168,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8407.35,0.0,188152.72,NaN,12015.54,0.0,198880.19,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905169,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12015.54,0.0,198880.19,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905170,14339972,TAXIFY ESTONIA OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14888.84,0.0,203471.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2275724,14339972,BOLT SERVICES EE OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
debtors_df = pd.read_excel('input/maksuvolglaste_nimekiri.xlsx', dtype={'Kood': str})
debtors_df = debtors_df.drop(['Nimi'], axis=1)
reg_df = pd.read_excel('input/Ъksuste nimekiri 01.12.2020_revid.4.10.2021.xlsx', dtype={'Registrikood': str})
reg_df = reg_df.drop(['Nimi'], axis=1)

result = pd.merge(df_merged,debtors_df,left_on=['Registrikood'],right_on=['Kood'], how='left')
result = pd.merge(result,reg_df,on=['Registrikood'], how='left')

# drop rows, where company created is before 2015
result = result[result['Üksuse registreerimise kuupäev'] >= pd.Timestamp('2015-01-01')]

Pankrotis/likvideeritud:

In [ ]:
# pankrotis: 
condition = list(map(result['Nimi'].str.contains, ['(PANKROTIS)']))

result['pankrotis'] = np.select(condition, [1], 0)
print(result['pankrotis'].value_counts())

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0    631226
1       409
Name: pankrotis, dtype: int64


In [ ]:
# likvideeritud: 
condition = list(map(result['Nimi'].str.contains, ['(LIKVIDEERIMISEL)']))

result['likvideeritud'] = np.select(condition, [1], 0)
print(result['likvideeritud'].value_counts())

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0    629611
1      2024
Name: likvideeritud, dtype: int64


In [ ]:
result.head()

,Registrikood,Nimi,Riiklikud Maksud_2017_i,Tööjõumaksud Ja Maksed_2017_i,Kaive_2017_i,Tootajaid_2017_i,Riiklikud Maksud_2017_ii,Tööjõumaksud Ja Maksed_2017_ii,Kaive_2017_ii,Tootajaid_2017_ii,Riiklikud Maksud_2017_iii,Tööjõumaksud Ja Maksed_2017_iii,Kaive_2017_iii,Tootajaid_2017_iii,Riiklikud Maksud_2017_iv,Tööjõumaksud Ja Maksed_2017_iv,Kaive_2017_iv,Tootajaid_2017_iv,Riiklikud Maksud_2018_i,Tööjõumaksud Ja Maksed_2018_i,Kaive_2018_i,Tootajaid_2018_i,Riiklikud Maksud_2018_ii,Tööjõumaksud Ja Maksed_2018_ii,Kaive_2018_ii,Tootajaid_2018_ii,Riiklikud Maksud_2018_iii,Tööjõumaksud Ja Maksed_2018_iii,Kaive_2018_iii,Tootajaid_2018_iii,Riiklikud Maksud_2018_iv,Tööjõumaksud Ja Maksed_2018_iv,Kaive_2018_iv,Tootajaid_2018_iv,Riiklikud Maksud_2019_i,Tööjõumaksud Ja Maksed_2019_i,Kaive_2019_i,Tootajaid_2019_i,Riiklikud Maksud_2019_ii,Tööjõumaksud Ja Maksed_2019_ii,Kaive_2019_ii,Tootajaid_2019_ii,Riiklikud Maksud_2019_iii,Tööjõumaksud Ja Maksed_2019_iii,Kaive_2019_iii,Tootajaid_2019_iii,Riiklikud Maksud_2019_iv,Tööjõumaksud Ja Maksed_2019_iv,Kaive_2019_iv,Tootajaid_2019_iv,Riiklikud Maksud_2020_i,Tööjõumaksud Ja Maksed_2020_i,Kaive_2020_i,Tootajaid_2020_i,Riiklikud Maksud_2020_ii,Tööjõumaksud Ja Maksed_2020_ii,Kaive_2020_ii,Tootajaid_2020_ii,Riiklikud Maksud_2020_iii,Tööjõumaksud Ja Maksed_2020_iii,Kaive_2020_iii,Tootajaid_2020_iii,Riiklikud Maksud_2020_iv,Tööjõumaksud Ja Maksed_2020_iv,Kaive_2020_iv,Tootajaid_2020_iv,Riiklikud Maksud_2021_i,Tööjõumaksud Ja Maksed_2021_i,Kaive_2021_i,Tootajaid_2021_i,Riiklikud Maksud_2021_ii,Tööjõumaksud Ja Maksed_2021_ii,Kaive_2021_ii,Tootajaid_2021_ii,Riiklikud Maksud_2021_iii,Tööjõumaksud Ja Maksed_2021_iii,Kaive_2021_iii,Tootajaid_2021_iii,Kood,Maksuvõlg,Sh vaidlustatud,Maksuvõlg alates,EMTAK2008 Tähtkoodiga valdkond,EMTAK2008 4-kohaline kood,Institutsionaalsete sektorite Eesti jaoks kohandatud klassifikaator 2010,Kliendi/emitendi identifikaator (vastavalt Eesti Panga presidendi 29. mai määrusele nr 7),Üksuse registreerimise kuupäev,Üksuse registrist kustutamise kuupäev,Muudatuse või lisamise kuupäev,pankrotis,likvideeritud
1273299,12775738,KJ PAIGALDUS OÜ,NaN,NaN,0.0,NaN,129.27,0.0,1272.67,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,2015-01-02,NaT,NaT,0,0
1273300,12775738,KJ PAIGALDUS OÜ,NaN,NaN,NaN,NaN,129.27,0.0,1272.67,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,2015-01-02,NaT,NaT,0,0
1273301,12775738,KJ PAIGALDUS OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,2015-01-02,NaT,NaT,0,0
1273302,12775738,KJ PAIGALDUS OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97

Labelling formula(s):

In [ ]:
# Average Increase:
def averageIncrease(list):
  changes = []
  for x1, x2 in zip(list[:-1], list[1:]):
      try:
          pct = (x2 - x1) * 100 / x1
      except ZeroDivisionError:
          pct = 0
      changes.append(pct)
  if len(changes) == 0:
    return 0
  avg = sum(changes)/len(changes)
  return avg

In [ ]:
# Compound Monthly Growth Rate - https://amplitude.com/blog/month-over-month-growth-rates
def CMGR(list):
  if (len(list) is 0):
    return 0
  return ((((list[len(list) - 1]/list[0]) ** (1/len(list))) - 1) * 100).real

In [ ]:
label_calc = result.filter(regex=("Kaive_.*"))
#label_calc['label'] = label_calc.apply(lambda x: x[0],axis=1)
label_calc['kaive_list'] = label_calc.values.tolist()
#remove nan values from list
label_calc['kaive_list'] = label_calc['kaive_list'].apply(lambda asd: [x for x in asd if math.isnan(x) == False and x != 0.0])
label_calc['averageIncrease'] = label_calc['kaive_list'].apply(lambda asd: averageIncrease(asd))
label_calc['compoundMonthlyGrowthRate'] = label_calc['kaive_list'].apply(lambda asd: CMGR(asd))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
result = pd.concat([result, label_calc['averageIncrease']], axis=1)
result = pd.concat([result, label_calc['compoundMonthlyGrowthRate']], axis=1)
result['label_avg'] = result['averageIncrease'].apply(lambda x: 'not fast-growing' if x <= 30 else 'fast-growing')
result['label'] = result['compoundMonthlyGrowthRate'].apply(lambda x: 'not fast-growing' if x <= 30 else 'fast-growing')

In [ ]:
result.head(5)

,Registrikood,Nimi,Riiklikud Maksud_2017_i,Tööjõumaksud Ja Maksed_2017_i,Kaive_2017_i,Tootajaid_2017_i,Riiklikud Maksud_2017_ii,Tööjõumaksud Ja Maksed_2017_ii,Kaive_2017_ii,Tootajaid_2017_ii,Riiklikud Maksud_2017_iii,Tööjõumaksud Ja Maksed_2017_iii,Kaive_2017_iii,Tootajaid_2017_iii,Riiklikud Maksud_2017_iv,Tööjõumaksud Ja Maksed_2017_iv,Kaive_2017_iv,Tootajaid_2017_iv,Riiklikud Maksud_2018_i,Tööjõumaksud Ja Maksed_2018_i,Kaive_2018_i,Tootajaid_2018_i,Riiklikud Maksud_2018_ii,Tööjõumaksud Ja Maksed_2018_ii,Kaive_2018_ii,Tootajaid_2018_ii,Riiklikud Maksud_2018_iii,Tööjõumaksud Ja Maksed_2018_iii,Kaive_2018_iii,Tootajaid_2018_iii,Riiklikud Maksud_2018_iv,Tööjõumaksud Ja Maksed_2018_iv,Kaive_2018_iv,Tootajaid_2018_iv,Riiklikud Maksud_2019_i,Tööjõumaksud Ja Maksed_2019_i,Kaive_2019_i,Tootajaid_2019_i,Riiklikud Maksud_2019_ii,Tööjõumaksud Ja Maksed_2019_ii,Kaive_2019_ii,Tootajaid_2019_ii,Riiklikud Maksud_2019_iii,Tööjõumaksud Ja Maksed_2019_iii,Kaive_2019_iii,Tootajaid_2019_iii,Riiklikud Maksud_2019_iv,Tööjõumaksud Ja Maksed_2019_iv,Kaive_2019_iv,Tootajaid_2019_iv,Riiklikud Maksud_2020_i,Tööjõumaksud Ja Maksed_2020_i,Kaive_2020_i,Tootajaid_2020_i,Riiklikud Maksud_2020_ii,Tööjõumaksud Ja Maksed_2020_ii,Kaive_2020_ii,Tootajaid_2020_ii,Riiklikud Maksud_2020_iii,Tööjõumaksud Ja Maksed_2020_iii,Kaive_2020_iii,Tootajaid_2020_iii,Riiklikud Maksud_2020_iv,Tööjõumaksud Ja Maksed_2020_iv,Kaive_2020_iv,Tootajaid_2020_iv,Riiklikud Maksud_2021_i,Tööjõumaksud Ja Maksed_2021_i,Kaive_2021_i,Tootajaid_2021_i,Riiklikud Maksud_2021_ii,Tööjõumaksud Ja Maksed_2021_ii,Kaive_2021_ii,Tootajaid_2021_ii,Riiklikud Maksud_2021_iii,Tööjõumaksud Ja Maksed_2021_iii,Kaive_2021_iii,Tootajaid_2021_iii,Kood,Maksuvõlg,Sh vaidlustatud,Maksuvõlg alates,EMTAK2008 Tähtkoodiga valdkond,EMTAK2008 4-kohaline kood,Institutsionaalsete sektorite Eesti jaoks kohandatud klassifikaator 2010,Kliendi/emitendi identifikaator (vastavalt Eesti Panga presidendi 29. mai määrusele nr 7),Üksuse registreerimise kuupäev,Üksuse registrist kustutamise kuupäev,Muudatuse või lisamise kuupäev,pankrotis,likvideeritud,averageIncrease,compoundMonthlyGrowthRate,label_avg,label
1273299,12775738,KJ PAIGALDUS OÜ,NaN,NaN,0.0,NaN,129.27,0.0,1272.67,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,2015-01-02,NaT,NaT,0,0,129.710188,17.957399,fast-growing,not fast-growing
1273300,12775738,KJ PAIGALDUS OÜ,NaN,NaN,NaN,NaN,129.27,0.0,1272.67,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,2015-01-02,NaT,NaT,0,0,129.710188,17.957399,fast-growing,not fast-growing
1273301,12775738,KJ PAIGALDUS OÜ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645.25,122.45,3464.0,NaN,447.04,452.02,2773.0,NaN,38.56,0.0,316.0,NaN,760.45,340.97,5099.5,2.0,6872.86,2824.38,25220.0,1.0,3527.96,2160.2,11506.5,1.0,2141.69,1837.38,4983.77,1.0,2120.43,1046.94,9887.57,1.0,4743.28,1782.66,22119.0,1.0,5627.06,3838.33,18203.02,1.0,3371.63,2294.19,10471.0,1.0,1028.14,525.98,5146.2,1.0,1511.23,889.48,8355.99,1.0,2542.32,1462.39,12033.95,1.0,1735.25,1396.4,8511.1,2.0,3308.35,2556.72,10930.91,2.0,6774.56,3488.56,24875.35,2.0,NaN,NaN,NaN,NaT,F,4329.0,S11002-S11003,8.0,20

In [ ]:
result['label_avg'].value_counts()

not fast-growing    452708
fast-growing        178927
Name: label_avg, dtype: int64

In [ ]:
result['label'].value_counts()

not fast-growing    579720
fast-growing         51915
Name: label, dtype: int64

Train, test, val stuff:⏰☹⛪


In [ ]:
# Find unique codes:
all_registrikoodid = result[['Registrikood']].values.ravel()
unique_registrikoodid =  pd.unique(all_registrikoodid)
#print(unique_registrikoodid)

# How many lines before duplicating rows:
original_data_len = len(unique_registrikoodid)

# Create test, train, val indexes:
train_data_indx = [0] * int(original_data_len * 0.6)
test_data_indx = [1] * int(original_data_len * 0.2)
val_data_indx= [2] * int(original_data_len * 0.2)

# Sum indexes and shuffle:
split_indx_array = train_data_indx + test_data_indx + val_data_indx
np.random.seed(42)
np.random.shuffle(split_indx_array)
#print(split_indx_array)

# make sure unique_registrikoodid is the same length as split_indx_array:
unique_registrikoodid = unique_registrikoodid[:len(split_indx_array)]

# Merge codes and test, train, val indexes:
d = {'Registrikoodid': unique_registrikoodid, 'Train_test_val': split_indx_array}
split_df = pd.DataFrame(data=d)

# Merge split_indx_array to original df:
result2 = pd.merge(result, split_df ,left_on=['Registrikood'], right_on=['Registrikoodid'], how='left')

# Convert categorical variables to binary:
result2 = pd.get_dummies(result2, columns = ['EMTAK2008 Tähtkoodiga valdkond', 'Institutsionaalsete sektorite Eesti jaoks kohandatud klassifikaator 2010'])

# Drop certain columns that models don't use and fill NAs:
result2 = result2.drop(columns=['Nimi','Üksuse registreerimise kuupäev','Registrikood', 'Registrikoodid', 'Kood', 'Sh vaidlustatud', 'Maksuvõlg alates', 'Muudatuse või lisamise kuupäev','Üksuse registrist kustutamise kuupäev', 'averageIncrease', 'compoundMonthlyGrowthRate', 'label_avg'])
result2 = result2.fillna(0)

# Create dataframes (test, train, val):
train_data = result2.loc[result2['Train_test_val'] == 0]
test_data = result2.loc[result2['Train_test_val'] == 1]
val_data = result2.loc[result2['Train_test_val'] == 2]

# Create labels:
train_data_labels = train_data['label'].replace('fast-growing', 1).replace('not fast-growing', 0)
test_data_labels = test_data['label'].replace('fast-growing', 1).replace('not fast-growing', 0)
val_data_labels = val_data['label'].replace('fast-growing', 1).replace('not fast-growing', 0)

train_data = train_data.drop(columns=['label' , 'Train_test_val'])
test_data = test_data.drop(columns=['label' , 'Train_test_val'])
val_data = val_data.drop(columns=['label' , 'Train_test_val'])


# Mudeldamine

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression()

np.random.seed(1111)

model_lr.fit(train_data, train_data_labels)

score_model_lr = model_lr.score(test_data, test_data_labels)
print(f'LogisticRegression accuracy is {score_model_lr*100}%')

LogisticRegression accuracy is 91.88342262069621%


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
model_etc = ExtraTreesClassifier(max_depth=10,n_jobs=16, n_estimators = 10)

np.random.seed(1111)

model_etc.fit(train_data, train_data_labels)
score_model_etc = model_etc.score(test_data, test_data_labels)
print(f'ExtraTreesClassifier accuracy is {score_model_etc*100}%')

{'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': 16, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
ExtraTreesClassifier accuracy is 91.94910224655973%


# **GridSearchCV🍩**

In [ ]:
# Grid search, 4 estimatori peal, https://www.davidsbatista.net/blog/2018/02/23/model_optimization/

class EstimatorSelectionHelper:
    
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, **grid_kwargs):
        for key in self.keys:
            print('Running GridSearchCV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            grid_search = GridSearchCV(model, params, **grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
        print('Done.')
    
    def score_summary(self, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [ ]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X_cancer = breast_cancer.data
y_cancer = breast_cancer.target

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

models1 = { 
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

params1 = { 
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
    'RandomForestClassifier': [
        { 'n_estimators': [16, 32] },
        {'criterion': ['gini', 'entropy'], 'n_estimators': [8, 16]}],
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] }
}

In [ ]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(train_data, train_data_labels, scoring='accuracy', n_jobs=2)


Running GridSearchCV for ExtraTreesClassifier.
Running GridSearchCV for RandomForestClassifier.
Running GridSearchCV for AdaBoostClassifier.
Running GridSearchCV for GradientBoostingClassifier.
Done.


In [ ]:
helper1.score_summary(sort_by='mean_test_score')

,estimator,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
0,RandomForestClassifier,42.332777,1.816905,0.963429,0.033697,{'n_estimators': 32},0.934990,0.929157,0.929130,0.940518,0.953079,0.937375,0.008918
1,RandomForestClassifier,21.841677,2.350343,0.488883,0.066759,"{'criterion': 'entropy', 'n_estimators': 16}",0.934304,0.928062,0.929091,0.937390,0.953488,0.936467,0.009169
2,RandomForestClassifier,20.957906,0.896738,0.494818,0.008853,"{'criterion': 'gini', 'n_estimators': 16}",0.934726,0.927903,0.927652,0.936863,0.952169,0.935863,0.008932
3,RandomForestClassifier,21.420509,1.168761,0.509910,0.032827,{'n_estimators': 16},0.933631,0.928457,0.926412,0.935728,0.952538,0.935353,0.009230
4,RandomForestClassifier,10.731367,0.594510,0.281317,0.012024,"{'criterion': 'gini', 'n_estimators': 8}",0.932509,0.925502,0.926201,0.934685,0.950255,0.933831,0.008942
5,RandomForestClassifier,11.563844,0.688436,0.272752,0.023331,"{'criterion': 'entropy', 'n_estimators': 8}",0.932324,0.926267,0.926333,0.936361,0.945835,0.933424,0.007285
6,GradientBoostingClassifier,83.279408,7.807220,0.172156,0.026131,"{'learning_rate': 1.0, 'n_estimators': 32}",0.927456,0.924829,0.929394,0.931281,0.912320,0.925056,0.006717
7,GradientBoostingClassifier,88.416027,5.424422,0.194047,0.010307,"{'learning_rate': 0.8, 'n_estimators': 32}",0.925793,0.928180,0.926848,0.933577,0.909655,0.924811,0.008038
8,AdaBoostClassifier,39.316788,1.636740,1.351267,0.158564,{'n_estimators': 32},0.917560,0.917901,0.918482,0.917466,0.916622,0.917606,0.000608
9,AdaBoostClassifier,21.099400,0.695987,0.807605,0.051633,{'n_estimators': 16},0.917388,0.917295,0.917572,0.917519,0.916476,0.917250,0.000399


## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(max_depth=10,n_jobs=16, n_estimators = 10)

np.random.seed(1111)

model_rfc.fit(train_data, train_data_labels)

score_model_rfc = model_rfc.score(test_data, test_data_labels)
print(f'model_rfc accuracy  is {score_model_rfc*100}%')

Average validation accuracy for model_rfc is 92.03852149622935%


## Bagged DT

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bagger = BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples = 0.8, max_features = 0.8, n_estimators=9, random_state=1111)

bagger.fit(train_data, train_data_labels)

score_bagger = bagger.score(test_data, test_data_labels)
print(f'BaggingClassifier accuracy is {score_bagger*100}%')

Average validation accuracy on BaggingClassifier is 96.01965640851145%


## XGBoost

In [ ]:
import xgboost as xgb

# XGBoosts wants data to be wrapped into special formats
dtrain = xgb.DMatrix(train_data, label=train_data_labels)
dtest = xgb.DMatrix(test_data, label=test_data_labels)

# most meaningful parameters
param_list = [("objective", "multi:softmax"), ("eval_metric", "merror"), ("num_class", 2)]

# Number of trees
n_rounds = 60 ## CHANGE IT TO 600

# if nothing seems to improve for 50 iterations - stop
early_stopping = 50

# train for training and test for ... validation!    
eval_list = [(dtrain, "train"), (dtest, "validation")]

# 1,2,3.. go!
bst = xgb.train(param_list, dtrain, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)

[0]	train-merror:0.074485	validation-merror:0.073617
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 50 rounds.
[1]	train-merror:0.070231	validation-merror:0.069177
[2]	train-merror:0.068436	validation-merror:0.067682
[3]	train-merror:0.067225	validation-merror:0.066946
[4]	train-merror:0.064407	validation-merror:0.064097
[5]	train-merror:0.064581	validation-merror:0.064168
[6]	train-merror:0.063884	validation-merror:0.063765
[7]	train-merror:0.064087	validation-merror:0.063986
[8]	train-merror:0.063414	validation-merror:0.063393
[9]	train-merror:0.061876	validation-merror:0.062016
[10]	train-merror:0.060337	validation-merror:0.060512
[11]	train-merror:0.059662	validation-merror:0.059697
[12]	train-merror:0.05821	validation-merror:0.058518
[13]	train-merror:0.057851	validation-merror:0.058233
[14]	train-merror:0.057511	validation-merror:0.058178
[15]	train-merror:0.056775	validation-merr

In [ ]:
ypred = bst.predict(dtest)
ypred = ypred.astype(int)
n_correct = np.sum(ypred == test_data_labels)
xgboost_accuracy = n_correct/len(test_data_labels)
print(f'Xgboost accuracy is {xgboost_accuracy}') # 0.9748992515831894

Xgboost accuracy is 0.9592548923408061


## Deep l

In [ ]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(111,)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              #optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
                   
history = model.fit(train_data,train_data_labels,epochs=20, batch_size=100, verbose=0, validation_split=0.1)

In [ ]:
# evaluate the model
scores = model.evaluate(test_data, test_data_labels)
print(f"Accuracy is {scores[1]*100}%")

3950/3950 [==============================] - 11s 3ms/step - loss: 0.2991 - accuracy: 0.9195
Accuracy is 91.94751977920532%


# Evaluation

TODO: xgboost ja deep l

In [ ]:
from sklearn import metrics
val_data = val_data.drop(['lr', 'etc', 'rfc', 'bagger', 'lr_pred', 'etc_pred', 'rfc_pred', 'bagger_pred'], axis=1)
N = val_data.shape[1]

val_data['lr'] = model_lr.predict_proba(val_data.iloc[:, : N])[:,1]
val_data['etc'] = model_etc.predict_proba(val_data.iloc[:, : N])[:,1]
val_data['rfc'] = model_rfc.predict_proba(val_data.iloc[:, : N])[:,1]
val_data['bagger'] = bagger.predict_proba(val_data.iloc[:, : N])[:,1]
val_data['lr_pred'] = model_lr.predict(val_data.iloc[:, : N])
val_data['etc_pred'] = model_etc.predict(val_data.iloc[:, : N])
val_data['rfc_pred'] = model_rfc.predict(val_data.iloc[:, : N])
val_data['bagger_pred'] = bagger.predict(val_data.iloc[:, : N])


# accuracy
print(f"LogisticRegression accuracy is {np.round(metrics.accuracy_score(val_data_labels, val_data['lr_pred']),3)}")
print(f"ExtraTreesClassifier accuracy is {np.round(metrics.accuracy_score(val_data_labels, val_data['etc_pred']),3)}")
print(f"RandomForestClassifier accuracy is {np.round(metrics.accuracy_score(val_data_labels, val_data['rfc_pred']),3)}")
print(f"BaggingClassifier desicion tree accuracy is {np.round(metrics.accuracy_score(val_data_labels, val_data['bagger_pred']),3)}")
print(f"Deep learn accuracy is {np.round(scores[1],3)}")
print(f'Xgboost accuracy is {np.round(xgboost_accuracy,3)}')

print("")
# AUC
print(f"AUC of LogisticRegression is {np.round(metrics.roc_auc_score(val_data_labels, val_data['lr']),3)}")
print(f"AUC of ExtraTreesClassifier is {np.round(metrics.roc_auc_score(val_data_labels, val_data['etc']),3)}")
print(f"AUC of RandomForestClassifier is {np.round(metrics.roc_auc_score(val_data_labels, val_data['rfc']),3)}")
print(f"AUC of BaggingClassifier desicion tree is {np.round(metrics.roc_auc_score(val_data_labels, val_data['bagger']),3)}")

print("")
# F1
print(f"F1 weighted of LogisticRegression is {np.round(metrics.f1_score(val_data_labels, val_data['lr_pred'] , average='weighted'),3)}")
print(f"F1 weighted of ExtraTreesClassifier is {np.round(metrics.f1_score(val_data_labels, val_data['etc_pred'] , average='weighted'),3)}")
print(f"F1 weighted of RandomForestClassifier is {np.round(metrics.f1_score(val_data_labels, val_data['rfc_pred'] , average='weighted'),3)}")
print(f"F1 weighted of BaggingClassifier desicion tree is {np.round(metrics.f1_score(val_data_labels, val_data['bagger_pred'] , average='weighted'),3)}")


LogisticRegression accuracy is 0.917
ExtraTreesClassifier accuracy is 0.917
RandomForestClassifier accuracy is 0.919
BaggingClassifier desicion tree accuracy is 0.959
Deep learn accuracy is 0.919
Xgboost accuracy is 0.959

AUC of LogisticRegression is 0.595
AUC of ExtraTreesClassifier is 0.634
AUC of RandomForestClassifier is 0.877
AUC of BaggingClassifier desicion tree is 0.955

F1 weighted of LogisticRegression is 0.879
F1 weighted of ExtraTreesClassifier is 0.878
F1 weighted of RandomForestClassifier is 0.881
F1 weighted of BaggingClassifier desicion tree is 0.955


# Old crap

In [ ]:
"""
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

test = result.drop(columns=["Registrikood"])
test = test.fillna(0)

scale= StandardScaler()
scaled_data = scale.fit_transform(test) 

kmeans = KMeans(n_clusters=3, random_state=0).fit(scaled_data)
clusters = kmeans.labels_
count_arr = np.bincount(clusters)
count_arr
"""

In [ ]:
"""
itemindex = np.where(clusters==2)
itemindex
"""

In [ ]:
"""
result.loc[821]
"""